In [1]:
import xml.dom.minidom
from xml.etree import ElementTree as ET
from bs4 import BeautifulSoup as bs

In [2]:
xmlfile1 = "./rezeptdatenbank.WordPress.2019-07-29_Rezepte.xml"
xmlfile2 = "./rezeptdatenbank.WordPress.2019-07-29_Rezeptev2.xml"

In [3]:
with open(xmlfile1, "r") as f:
    content = f.read()

In [4]:
content = content.replace('\t','')
content = content.replace('<content:encoded>\n','<content:encoded>')
content = content.replace('<excerpt:encoded>\n','<excerpt:encoded>')
content = content.replace('<title>\n','<title>')
content = content.replace('</item>','</item>\n')
content = content.replace('<wp:author>','<wp:author>\n')
content = content.replace('</wp:author>','</wp:author>\n')
content = content.replace('</wp:author_id>','</wp:author_id>\n')
content = content.replace('</wp:author_login>','</wp:author_login>\n')
content = content.replace('</wp:author_email>','</wp:author_email>\n')
content = content.replace('</wp:author_display_name>','</wp:author_display_name>\n')
content = content.replace('</wp:author_first_name>','</wp:author_first_name>\n')
content = content.replace('</wp:author_last_name>','</wp:author_last_name>\n')

In [5]:
with open(xmlfile2, "w") as f:
    f.write(content)

In [6]:
def find_recipe(text):
    pos1 = text.find('<item>')
    pos2 = text.find('</item>')+7
    return pos1, pos2

In [7]:
pos1 = 0
text = content
recipes = []
while pos1 != -1:
    pos1, pos2 = find_recipe(text)
    recipes.append(text[pos1:pos2])
    text =  text[pos2+1:]

In [8]:
len(recipes)

7037

In [9]:
recipes_notnull = [r for r in recipes if '<content:encoded><![CDATA[]]></content:encoded>' not in r]

In [10]:
len(recipes_notnull)

6893

In [11]:
#recipes_notnull[0]

In [12]:
header = content[0:content.find('<item>')]
footer = '''
</channel>
</rss>
'''

In [14]:
for idn, r in enumerate(recipes_notnull):
    with open(f'./data/recipe_{idn}.xml', "w") as f:
        f.write(header + r + footer)

#### Alternative ways to navigate the contents

In [ ]:
soup = bs(content, "lxml")
#result = soup.find_all('content:encoded')
result = soup.find('item')
print(len(list(result)))

In [ ]:
doc = xml.dom.minidom.parse(xmlfile)
tree = ET.parse(xmlfile)

In [ ]:
recipes = doc.getElementsByTagName("item")
len(recipes)

In [ ]:
root = tree.getroot()
print(root)

In [ ]:
for child in root:
    print(child.tag, child.attrib)

In [ ]:
set([elem.tag for elem in root.iter()])

In [ ]:
for event, elem in ET.iterparse(xmlfile):
    if elem.tag == "item":
        print(repr(elem.findtext("link")))
        elem.clear() # won't need this again
        break

In [ ]:
for event, elem in ET.iterparse(xmlfile):
    if elem.tag == "title":
        print repr(elem.text)
        break # we're done